In [40]:
from dotenv import load_dotenv
import os
import requests

load_dotenv()

api_key = os.environ.get('riot_api_key')


def get_puuid(gameName = None, tagLine = None, api_key = None):

    link = f'https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{gameName}/{tagLine}?api_key={api_key}'

    response = requests.get(link)

    return response.json()['puuid']


def get_name_and_tag(puuid=None, api_key=None):
    link = f'https://americas.api.riotgames.com/riot/account/v1/accounts/by-puuid/{puuid}?api_key={api_key}'

    response = requests.get(link)

    gameName = response.json()['gameName']
    tagLine = response.json()['tagLine']

    return f'{gameName}#{tagLine}'

In [ ]:
get_puuid(gameName='Kleny', tagLine='BR1', api_key=api_key)

In [ ]:
get_name_and_tag(puuid='vDP4mFBvhEaaet227PMfXLZNffp6mDLlZ3vPuSiweOJB6XbqECsqiXPKDYaFM4S1nVFp73G4LNHEtw', api_key=api_key)